# Distributed Alignment Search

In [1]:
from typing import Any

import torch
import nnsight
from nnsight import AbstractModel, LanguageModel, util
# from nnsight.Module import Module
Module = nnsight.Module
from nnsight.toolbox.optim.lora import LORA
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset as hf_Dataset
from das_utils import factual_sampler

# If using llama 7b
# from huggingface_hub import notebook_login
# notebook_login()

In [4]:
model = LanguageModel('EleutherAI/pythia-6.9b', device_map="cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
n_tokens = 10
epochs = 1
answer = "Paris"
answer_tokens = model.tokenizer(answer)
answer_token = answer_tokens["input_ids"][0]

lora = LORA(model.transformer.h[0].mlp, 10)

optimizer = torch.optim.AdamW(lora.parameters(), lr=.1)
dataset = [[" ".join(["_"] * n_tokens), answer_token]] * 100
dataloader = DataLoader(dataset, batch_size=10)


lossfn = util.cross_entropy_loss

In [16]:
for epoch in range(epochs):
    print(epoch)

    for i, (inputs, targets) in enumerate(dataloader):
        print(f"  {i}")

        optimizer.zero_grad()

        with model.forward(inference=False) as runner:
            with runner.invoke(inputs) as invoker:

                lora()
    
                logits = model.lm_head.output.save()

        print(lora.WA)
        loss = lossfn(logits.value[:, -1], targets)
        print(loss)

        loss.backward()

        optimizer.step()

0
  0
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], requires_grad=True)
tensor(nan, grad_fn=<MeanBackward1>)
  1
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], requires_grad=True)
tensor(nan, grad_fn=<MeanBackward1>)
  2
Parameter containing:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ...,

In [23]:
with model.generate() as generator:
    with generator.invoke(dataset[0][0]) as invoker:
        pass

print(model.tokenizer.decode(generator.output[0]))


with model.generate() as generator:
    with generator.invoke(dataset[0][0]) as invoker:
        lora()

print(model.tokenizer.decode(generator.output[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_ _ _ _ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _ _ _!


In [5]:
raw_prealign = factual_sampler(model.tokenizer, 5000, game="pricing_tag")
prealign_dataset = hf_Dataset.from_dict(
    {"input_ids": raw_prealign[0], "labels": raw_prealign[1]})
prealign_dataset.set_format('torch', columns=['input_ids','labels'])
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [6]:
prealign_input_batches = torch.split(prealign_dataset["input_ids"], int(5000/100), dim=0)
prealign_labels_batches = torch.split(prealign_dataset['labels'][:,-1], int(5000/100), dim=0)

In [16]:
model.tokenizer.decode(raw_prealign[0][4])

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPlease say yes only if it costs between 0.25 and 5.13 dollars, otherwise no.\n\n### Input:\n9.02 dollars\n\n### Response:\n'

In [17]:
with model.generate(max_new_tokens=1) as generator:
    with generator.invoke(raw_prealign[0][4]) as invoker:
        pass

out = generator.output[:,-1:].cpu()
model.tokenizer.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'\n'

In [9]:
per_batch_acc = []
per_batch_res = []
total_correct = 0

for i, batch in enumerate(prealign_input_batches):
    with model.generate(max_new_tokens=1) as generator:
        with generator.invoke(batch) as invoker:
            pass
            
    out = generator.output[:,-1].cpu()
    per_batch_res.append(out)
    correct = (out==prealign_labels_batches[0]).sum()
    per_batch_acc.append(correct)
    total_correct += (correct)
    

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


KeyboardInterrupt: 

In [ ]:
t_total = int(len(train_dataloader) * 3)
warm_up_steps = 0.1 * t_total
optimizer_params = []
for k, v in alignable.interventions.items():
    optimizer_params += [{'params': v[0].rotate_layer.parameters()}]
    optimizer_params += [{'params': v[0].intervention_boundaries, 'lr': 1e-2}]
optimizer = torch.optim.Adam(
    optimizer_params,
    lr=1e-3
)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warm_up_steps,
    num_training_steps=t_total
)

# You can define your custom compute_metrics function.
def compute_metrics(eval_preds, eval_labels):
    total_count = 0
    correct_count = 0
    for eval_pred, eval_label in zip(eval_preds, eval_labels):
        actual_test_labels = eval_label[:, -1]
        pred_test_labels = torch.argmax(eval_pred[:, -1], dim=-1)
        correct_labels = (actual_test_labels==pred_test_labels)
        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
    accuracy = round(correct_count/total_count, 2)
    return {"accuracy" : accuracy}

epochs = 3
gradient_accumulation_steps = 4
total_step = 0
target_total_step = len(train_dataloader) * epochs
temperature_start = 50.0
temperature_end = 0.1
temperature_schedule = torch.linspace(
    temperature_start, temperature_end, target_total_step
).to(torch.bfloat16).to("cuda")

# is this correct
alignable.set_temperature(temperature_schedule[total_step])

def calculate_loss(logits, labels):
    shift_logits = logits[..., :, :].contiguous()
    shift_labels = labels[..., :].contiguous()
    # Flatten the tokens
    loss_fct = CrossEntropyLoss()
    shift_logits = shift_logits.view(-1, alignable.model_config.vocab_size)
    shift_labels = shift_labels.view(-1)
    # Enable model parallelism
    shift_labels = shift_labels.to(shift_logits.device)
    loss = loss_fct(shift_logits, shift_labels)
    
    for k, v in alignable.interventions.items():
        boundary_loss = 1. * v[0].intervention_boundaries.sum()
    loss += boundary_loss
    
    return loss